# Apply pipelines
We combine *preprocessing* and *classification* into a pipeline, then perform grid search to find the best classifier in the *classification* stage of the pipeline.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import mglearn

## 1. Revisiting adult dataset from week 5 - Encoding example

In [3]:
import os
# The file has no headers naming the columns, so we pass header=None
# and provide the column names explicitly in "names"
adult_path = os.path.join(mglearn.datasets.DATA_PATH, "adult.data")
data = pd.read_csv(
    adult_path, header=None, index_col=False,
    names=['age', 'workclass', 'fnlwgt', 'education',  'education-num',
           'marital-status', 'occupation', 'relationship', 'race', 'gender',
           'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
           'income'])
# For illustration purposes, we only select some of the columns
data = data[['age', 'workclass', 'education', 'gender', 'hours-per-week',
             'occupation', 'income']]
# IPython.display allows nice output formatting within the Jupyter notebook
display(data.head())

,age,workclass,education,gender,hours-per-week,occupation,income
0,39,State-gov,Bachelors,Male,40,Adm-clerical,<=50K
1,50,Self-emp-not-inc,Bachelors,Male,13,Exec-managerial,<=50K
2,38,Private,HS-grad,Male,40,Handlers-cleaners,<=50K
3,53,Private,11th,Male,40,Handlers-cleaners,<=50K
4,28,Private,Bachelors,Female,40,Prof-specialty,<=50K


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   education       32561 non-null  object
 3   gender          32561 non-null  object
 4   hours-per-week  32561 non-null  int64 
 5   occupation      32561 non-null  object
 6   income          32561 non-null  object
dtypes: int64(2), object(5)
memory usage: 1.7+ MB


### 1.1 Build the ColumnTransformer
**Transform by column name**

In [5]:
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import StandardScaler, OneHotEncoder

ct = ColumnTransformer(
    [("scaling", StandardScaler(), ['age', 'hours-per-week']),
     ("onehot", OneHotEncoder(sparse_output=False), ['workclass', 'education', 'gender', 'occupation'])])

**We can select columns by data type too.**

In [6]:
ct = ColumnTransformer(
    [("scaling", StandardScaler(), make_column_selector(dtype_exclude=object)),
     ("onehot", OneHotEncoder(sparse_output=False), make_column_selector(dtype_include=object))])

### 1.2 Build the Pipeline

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

pipe = Pipeline(steps=[('preprocessor', ct),
                      ('classifier', LogisticRegression(max_iter=1000))])

In [8]:
from sklearn import set_config
set_config(display='diagram')
pipe

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('scaling', StandardScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x169baaa90>),
                                                 ('onehot',
                                                  OneHotEncoder(sparse_output=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x169baaac0>)])),
                ('classifier', LogisticRegression(max_iter=1000))])

### 1.3 Split the data

In [9]:
from sklearn.model_selection import train_test_split
# get all columns apart from income for the features
data_features = data.drop("income", axis=1)
# split dataframe and income
X_train, X_test, y_train, y_test = train_test_split(
    data_features, data.income, random_state=0)



### 1.4 Train the pipeline and assess on training set


In [10]:
pipe.fit(X_train, y_train)

print("Training score: {:.2f}".format(pipe.score(X_train, y_train)))

Training score: 0.81


### 1.5 Access pipeline steps

In [11]:
set_config(display='diagram')
pipe.named_steps.preprocessor

ColumnTransformer(transformers=[('scaling', StandardScaler(),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x169baaa90>),
                                ('onehot', OneHotEncoder(sparse_output=False),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x169baaac0>)])

In [12]:
set_config(display='diagram')
pipe['preprocessor']

ColumnTransformer(transformers=[('scaling', StandardScaler(),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x169baaa90>),
                                ('onehot', OneHotEncoder(sparse_output=False),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x169baaac0>)])

In [13]:
set_config(display='text')
pipe['preprocessor'].transformers_[1][1].get_feature_names_out(['workclass', 'education', 'gender', 'occupation'])

array(['workclass_ ?', 'workclass_ Federal-gov', 'workclass_ Local-gov',
       'workclass_ Never-worked', 'workclass_ Private',
       'workclass_ Self-emp-inc', 'workclass_ Self-emp-not-inc',
       'workclass_ State-gov', 'workclass_ Without-pay',
       'education_ 10th', 'education_ 11th', 'education_ 12th',
       'education_ 1st-4th', 'education_ 5th-6th', 'education_ 7th-8th',
       'education_ 9th', 'education_ Assoc-acdm', 'education_ Assoc-voc',
       'education_ Bachelors', 'education_ Doctorate',
       'education_ HS-grad', 'education_ Masters', 'education_ Preschool',
       'education_ Prof-school', 'education_ Some-college',
       'gender_ Female', 'gender_ Male', 'occupation_ ?',
       'occupation_ Adm-clerical', 'occupation_ Armed-Forces',
       'occupation_ Craft-repair', 'occupation_ Exec-managerial',
       'occupation_ Farming-fishing', 'occupation_ Handlers-cleaners',
       'occupation_ Machine-op-inspct', 'occupation_ Other-service',
       'occupation_ P

In [14]:
pipe.named_steps.classifier.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 1000,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

### 1.6 Grid search
We use the named step in the pipeline, followed by two underscores, then the hyper-parameter name to define the grid parameters:

In [15]:
from sklearn.model_selection import GridSearchCV

param_grid = {'classifier__C': [0.01, 0.1, 1.0, 10.0],
             'classifier__fit_intercept': [True, False]}
grid = GridSearchCV(pipe, param_grid, cv=5)


In [16]:
grid.fit(X_train, y_train)

print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))

Best params:
{'classifier__C': 1.0, 'classifier__fit_intercept': True}

Best cross-validation score: 0.81
Test-set score: 0.81
